# Recurrent Neural Network - RNN (Yinelemeli Sinir Agi)

## Normalization:
CIFAR-10 ve CIFAR-100:
Kanal Sayısı: 3 (RGB)
Normalizasyon: transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

MNIST:
Kanal Sayısı: 1 (Grayscale)
Normalizasyon: transforms.Normalize((0.5,), (0.5,))

MS-COCO:
Kanal Sayısı: 3 (RGB)
Normalizasyon: transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

ImageNet:
Kanal Sayısı: 3 (RGB)
Normalizasyon: Genellikle ImageNet veri seti için kullanılan normalizasyon parametreleri: transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

Fashion-MNIST:
Kanal Sayısı: 1 (Grayscale)
Normalizasyon: transforms.Normalize((0.5,), (0.5,))

In [1]:
import torch
import torch.nn as nn
from torch.optim import RMSprop
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # her kanalin ort. ve std(stand. sapma) degerini -> 0.5
])
"""
Normalize icini goruntu kac kanalli ise o kanal sayisi kadar doldurmamiz gerekli.
Ornegin, MNIST; gray scale(siyah-beyaz) old. icin sadece Normalize((0.5), (0.5)) yeterli olurken,
CIFAR10-CIFAR100; RGB oldugundan Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) olur.
R-Read, G-Green, B-Blue
transforms.Normalize((mean1, mean2), (std1, std2))
transforms.Normalize((mean_R, mean_G, mean_B), (std_R, std_G, std_B))
"""

In [4]:
train = MNIST(root='', train=True, download=False, transform=transform)
test = MNIST(root='', train=False, download=False, transform=transform)

In [5]:
batch_size = 100

train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [6]:
test_data_loader.dataset.extra_repr

<bound method MNIST.extra_repr of Dataset MNIST
    Number of datapoints: 10000
    Root location: 
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )>

In [35]:
class RNNModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        """
        input_dim: Giriş verisinin boyutu. Örneğin, bir zaman serisi veri setinde her bir zaman adımındaki özellik sayısı.
        hidden_dim: Gizli durumun boyutu. RNN'nin içsel gizli durumunun boyutu, yani ağın hatırlama kapasitesini belirler.
        layer_dim: RNN'nin kaç katmanlı olduğu. Bir RNN'nin tek bir katmanı, birbirine bağlı bir dizi hücreden oluşur. Eğer bu değer 1 ise, tek bir katmanlı bir RNN olur. Ancak, 2 veya daha fazla bir değerse, bu RNN'nin kaç katmanlı olduğunu belirler. Daha derin (katmanlı) bir yapı, daha karmaşık öğrenme yetenekleri sunabilir, ancak daha fazla hesaplama maliyetiyle birlikte gelir.
        output_dim: RNN'nin çıkışının boyutu. Örneğin, sınıflandırma problemlerinde çıkış sınıf sayısı olabilir.
        
        Giriş boyutu (input_dim): Ağa ne tür bir veri beslediğimizi belirler. Girdi özellik sayısı.
        Gizli durum boyutu (hidden_dim): Ağın ne kadar karmaşık bilgileri hatırlayabileceğini belirler. Gizli katmandaki nöron sayısı.
        Katman sayısı (layer_dim): Ağın kaç katmanlı olduğunu ve ne kadar karmaşık öğrenme yapabileceğini belirler. RNN katman sayısı.
        Çıkış boyutu (output_dim): Ağın ne tür bir çıkış verisi üreteceğini belirler. Çıkış özellik sayısı.
        
        Katman sayısı (layer_dim) özellikle ilginç bir parametredir çünkü çok katmanlı RNN'ler, daha karmaşık bağlantıları öğrenme yeteneği ile öne çıkabilir. Ancak, bu aynı zamanda eğitim sürecini zorlaştırabilir ve daha fazla hesaplama maliyeti gerektirebilir. Bu parametre, modelin karmaşıklığını ve performansını belirleyen önemli bir kademeli parametredir.
        """
        
        # Number of Hidden Dimensions
        self.hidden_dim = hidden_dim # class'in her yerinden erismek icin boyle yaptik
        
         # Number of Hidden Layers
        self.layer_dim = layer_dim # class'in her yerinden erismek icin boyle yaptik
        
        # Recurrent Neural Network - RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, bias=True, dropout=0.5,
                          nonlinearity='relu', batch_first=True, bidirectional=True)
        """
        input_dim: Giriş verisinin boyutu. Her bir zaman adımındaki özellik sayısını belirtir.
        hidden_dim: Gizli durumun boyutu. RNN'nin içsel gizli durumunun boyutunu belirler.
        layer_dim: RNN'nin kaç katmanlı olduğu. Eğer bu değer 1 ise, tek katmanlı bir RNN olur. Ancak, 2 veya daha fazla bir değerse, bu RNN'nin kaç katmanlı olduğunu belirler.
        nonlinearity: RNN hücresinde kullanılacak aktivasyon fonksiyonunu belirler. Varsayılan değer "tanh"tır.
        bias: Modelin öğrenme sırasında öğrenilen sapma terimini (bias) kullanıp kullanmayacağını belirler. Varsayılan değer True'dir.
        batch_first: Eğer True ise, giriş verisi şekli (batch, time, input_dim) olur; False ise (time, batch, input_dim). Varsayılan değer False'dir. Yani, her bir mini-batch'in başında batch boyutunu belirtmek isteriz.
        dropout: Eğitim sırasında kullanılacak dropout katmanının olasılığını belirler. Varsayılan değer 0'dır, yani dropout kullanılmaz.
        bidirectional: Eğer True ise, çift yönlü bir RNN oluşturur. Bu, hem ileri hem de geri yönde birbiri ardına iki RNN'yi birleştirir. Varsayılan değer False'dir.
        device: Tensorların nerede depolanacağını belirler. Örneğin, torch.device("cuda") veya "cpu" olabilir.
        dtype: Tensorların veri tipini belirler. Örneğin, torch.float veya torch.double olabilir.
        """
        
        # Readout Layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # bidirectional verdiğimizde(çift yönlülük) giris ve cikis olacagi icin hidden_dim * 2 yapariz!
        """
        "Readout Layer" olarak adlandırılan bu katman, RNN'ın çıktılarını alır ve istediğiniz çıkış boyutuna dönüştürür. Bu katman, genellikle RNN'ın çıktılarından öğrenilen özellikleri temsil eder ve bu özellikleri kullanarak belirli bir görevi gerçekleştirmek üzere eğitilir.
        
        nn.Linear: Bu, tam bağlantılı (fully connected) bir katman oluşturur. Yani, bu katmanın tüm giriş birimleri (neurons) tüm çıkış birimleriyle bağlıdır.
        hidden_dim: Bu, gelen RNN çıktılarının boyutunu temsil eder. RNN katmanından gelen her çıkış, bu boyuta sahip bir vektördür. Giriş özellik sayısı(RNN çıkışının özellik sayısı).
        output_dim: Bu, bu tam bağlantılı katmanın çıkış boyutunu temsil eder. Yani, bu katmanın kaç adet çıkış birimine sahip olduğunu belirtir. Çıkış özellik sayısı.
        
        Bu katmanın temel görevi, RNN tarafından öğrenilen özellikleri, probleminize uygun bir çıkışa dönüştürmektir. Örneğin, bir dil modeli eğitiyorsanız, çıkış boyutu muhtemelen kelime dağarcığınızın boyutuna eşit olacaktır.
        
        Bu, genellikle bir sınıflandırma problemi için kullanılan bir yapıdır. Eğitim sırasında, RNN'dan gelen çıktılar bu lineer katmana beslenir ve ardından bir softmax aktivasyon fonksiyonu ile geçirilerek olasılık dağılımını elde ederiz. Bu olasılık dağılımı, modele verilen girdi dizisinin hangi sınıfa ait olduğunu belirlemek için kullanılır.
        """
    
    def forward(self, x):
        """
        forward fonksiyonu bir RNN'in bir geçişini tanımlar.
        İlk olarak, gizli durumu sıfırlar, ardından her bir zaman adımını çalıştırır ve sadece en son çıkışı kullanarak bir çıkış üretir.
        Bu, tipik bir RNN'nin ileri geçişini temsil eder.
        """
        
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim * 2, x.size(0), self.hidden_dim).to(device) # bidirectional(çift yönlülük) verdiğimizde giris ve cikis olacagi icin layer_dim * 2 yapariz!
        """
        İlk olarak, gizli durumu sıfırlarız (h0), bu durumu ilk gizli durum olarak kullanmak üzere tanımlarız.
        self.layer_dim kaç tane RNN katmanı kullanılacağını belirtir. Eğer sadece tek bir katman kullanacaksak, bu değer genellikle 1 olacaktır.
        x.size(0) gelen veri yığınının boyutu, yani batch size'tır.
        self.hidden_dim ise gizli durumun boyutunu ifade eden bir tensör oluşturulur.
        """
        
        # One time step
        out, hn = self.rnn(x, h0)
        """
        Ardından, self.rnn (nn.RNN katmanı) ile bir zaman adımını çalıştırırız.
        out çıkış tensörü, her bir zaman adımındaki çıkışları içerir.
        hn ise final gizli durumu içerir.
        """
        out = self.fc(out[:, -1, :])
        """
        Son olarak, sadece en son zaman adımının çıkışını kullanarak bir tam bağlantılı katmandan geçiririz.
        out[:, -1, :] ifadesi, her bir örneğin en son zaman adımındaki çıkışları alır.
        İlk boyut (:): Tüm batch boyutu. Her bir örnek için aynı işlemi yapmak istiyoruz.
        İkinci boyut (-1): Son zaman adımını seçer. -1 ifadesi, Python'da sondan bir önceki elemanı ifade eder. Bu durumda, RNN'nin tüm zaman adımları arasından sadece son zaman adımını seçer.
        Üçüncü boyut (:): Tüm özellikler. Her bir çıkışın tüm özelliklerini korumak istiyoruz.
        
        Yani, bu indexing ifadesiyle out[:, -1, :], RNN'nin çıkış tensoründen tüm örneklerin son zaman adımını ve tüm özellikleri alır.
        RNN'nin çalışma mantığı göz önüne alındığında:
        RNN'nin çıkış tensoru genellikle şu şekildedir: (batch_size, sequence_length, hidden_size)
        [:, -1, :] ifadesi bu tensörün tüm batch'lerini, son zaman adımını ve tüm özelliklerini seçer.
        """
        
        return out

In [47]:
input_dim = 28
hidden_dim = 128
layer_dim = 3
output_dim = 10

In [48]:
model = RNNModel(input_dim=input_dim,
                hidden_dim=hidden_dim,
                layer_dim=layer_dim,
                output_dim=output_dim).to(device)

In [49]:
criterion = nn.CrossEntropyLoss()

lr = 5e-4
optimizer = RMSprop(model.parameters(), lr=lr, momentum=0.3)

In [50]:
num_epoch = 30

for epoch in range(num_epoch):
    for images, labels in train_data_loader:
        images, labels = images.view(-1, 28, 28).to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch+1}/{num_epoch} /// Loss: {loss.item()}')

Epoch: 1/30 /// Loss: 0.4188695549964905
Epoch: 2/30 /// Loss: 0.2226596474647522
Epoch: 3/30 /// Loss: 0.06615384668111801
Epoch: 4/30 /// Loss: 0.138435959815979
Epoch: 5/30 /// Loss: 0.12217563390731812
Epoch: 6/30 /// Loss: 0.1830991506576538
Epoch: 7/30 /// Loss: 0.14898836612701416
Epoch: 8/30 /// Loss: 0.14905282855033875
Epoch: 9/30 /// Loss: 0.03261500597000122
Epoch: 10/30 /// Loss: 0.05956607684493065
Epoch: 11/30 /// Loss: 0.05525694414973259
Epoch: 12/30 /// Loss: 0.013808483257889748
Epoch: 13/30 /// Loss: 0.09418260306119919
Epoch: 14/30 /// Loss: 0.08788786828517914
Epoch: 15/30 /// Loss: 0.16557855904102325
Epoch: 16/30 /// Loss: 0.07329320162534714
Epoch: 17/30 /// Loss: 0.03444778546690941
Epoch: 18/30 /// Loss: 0.048056941479444504
Epoch: 19/30 /// Loss: 0.005227099638432264
Epoch: 20/30 /// Loss: 0.02537042647600174
Epoch: 21/30 /// Loss: 0.08549755066633224
Epoch: 22/30 /// Loss: 0.013520748354494572
Epoch: 23/30 /// Loss: 0.030250828713178635
Epoch: 24/30 /// Los

In [51]:
model.eval()
correct = 0
total = 0

In [52]:
with torch.no_grad():
    for image, label in test_data_loader:
        image = image.view(-1, 28, 28).to(device)
        label = label.to(device)
        
        output = model(image)
        predicted = torch.max(output.data, 1)[1]
        total += label.size(0)
        correct += (predicted == label).sum().item()

In [53]:
accuracy = (correct / total) * 100
print('Accuracy: {0}'.format(accuracy))

Accuracy: 98.68


## Tek Image vererek tahmin fonksiyonumuz

In [54]:
def predictSingleImage(model, image):
    model.eval()
    image = image.view(-1, 28, 28).to(device)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    
    return predicted.item()

In [55]:
sample_image_single, sample_label_single = next(iter(test_data_loader))

predicted_single_image = predictSingleImage(model, sample_image_single[0])

print(f'Predict: {predicted_single_image}, Real Class: {sample_label_single[0].item()}')

Predict: 7, Real Class: 7


## Birden cok Image vererek tahmin fonksiyonumuz

In [56]:
def predictImages(model, images):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for image in images:
            image = image.view(-1, 28, 28).to(device)
            output = model(image)
            predict = torch.max(output.data, 1)[1]
            predictions.append(predict.item())
    
    return predictions

In [57]:
sample_images, sample_labels = next(iter(test_data_loader))

predicted_images = predictImages(model, sample_images[:10])

for i, (predict_class, true_label) in enumerate(zip(predicted_images, sample_labels[:10])):
    print(f'Sampler {i+1} - Predict {predict_class}, Real Class: {true_label.item()}')

Sampler 1 - Predict 7, Real Class: 7
Sampler 2 - Predict 2, Real Class: 2
Sampler 3 - Predict 1, Real Class: 1
Sampler 4 - Predict 0, Real Class: 0
Sampler 5 - Predict 4, Real Class: 4
Sampler 6 - Predict 1, Real Class: 1
Sampler 7 - Predict 4, Real Class: 4
Sampler 8 - Predict 9, Real Class: 9
Sampler 9 - Predict 5, Real Class: 5
Sampler 10 - Predict 9, Real Class: 9
